In [1]:
from sklearn import metrics, random_projection, cross_validation, datasets, grid_search, linear_model, metrics, random_projection, manifold, preprocessing
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import  Counter
import numpy as np
import pandas as pd
from sklearn.decomposition import RandomizedPCA
from sklearn import manifold
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.svm import SVC, OneClassSVM
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
from scipy import load, save
import sklearn
%pylab inline

/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/tosha/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
dataframe = pd.read_csv('dataframe/genres_40.csv')
dataframe.loc[:, 'm1':] = preprocessing.normalize(dataframe.loc[:, 'm1':])

labels = ['hiphop', 'neoclassic', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
labels_n = ['neoclassic', 'jazz',  'metal', 'pop']

# dataframe = dataframe[dataframe['tag'] == 'neoclassic'].append(dataframe[dataframe['tag'] == 'jazz']).append(dataframe[dataframe['tag'] == 'pop']).append(dataframe[dataframe['tag'] == 'metal'])
# dataframe.append(dataframe[dataframe['tag'] == 'pop'])
# dataframe = dataframe.append(dataframe[dataframe['tag'] == 'jazz'])
# dataframe = dataframe.append(dataframe[dataframe['tag'] == 'metal'])
dataframe = dataframe.sample(frac=1)

shape(dataframe)

(1601, 201)

In [3]:
def max_cl(arr):
    list_max_i = []
    for a in arr:
        max_el = 0
        max_i = 0
        for i in range(len(a)):
            if a[i] > max_el:
                max_el = a[i]
                max_i = i
        list_max_i += [max_i]
    return list_max_i

max_cl([[1,2,3], [2,6,4], [8,1,2]])

[2, 1, 0]

In [4]:
def search_abnom(dataframe, label, g, nu):
    svm_ = OneClassSVM(gamma=g, nu=nu)
    svm_.fit(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    l = svm_.predict(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    return l

labels = ['hiphop', 'classical', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
for label in labels:
    svm_label = search_abnom(dataframe, label, 2, 0.15).tolist()
    dataframe[dataframe['tag'] == label] = dataframe[dataframe['tag'] == label][np.array(svm_label) == 1]
    dataframe = dataframe.dropna()
print(shape(dataframe))

(1350, 201)


In [5]:
label_dict = {'hiphop':0, 'classical':1, 'jazz':2, 'rock':3, 'metal':4, 'synth':5, 'pop':6, 'blues': 7} 
int_label = []
for label in dataframe['tag']:
    int_label.append(label_dict[label])

In [6]:
# svm_label = []
# for label in labels:
#     svm_label += search_abnom(label, 2, 0.01).tolist()
# dataframe_svm = dataframe[np.array(svm_label) == 1]

# dataframe_svm_t = np.array(dataframe_svm.loc[:, 'm1':])
# dataframe_svm_l = np.array(dataframe_svm['tag'])

dataframe_svm_t = np.array(dataframe.loc[:, 'm1':])
dataframe_svm_l = np.array(int_label)
dataframe_t = dataframe.loc[:, 'm1':]
np.shape(dataframe_svm_l)

(1350,)

In [7]:
svcc = SVC(C=10, gamma=5, kernel = 'rbf', probability=True)
lrc = LogisticRegression(C = 430)
knc = KNeighborsClassifier(weights = 'distance', n_neighbors=7, metric = 'manhattan')
rfc = RandomForestClassifier(max_depth=23, n_estimators=1300, random_state=1)
xgbc = xgb.XGBClassifier(silent=False, nthread=4, max_depth=2, n_estimators=4000, learning_rate=0.009)

In [65]:
def fin_classif(list_model, list_param, list_name):
    list_preds_f = []
    list_labels_f = []
    for train_ind, test_ind in cross_validation.StratifiedKFold(dataframe_svm_l, n_folds=5, shuffle=True, 
                                                                random_state=1):
        np.random.shuffle(train_ind) 
        np.random.shuffle(test_ind)

#         f_df = dataframe_svm_t[:, :]
#         s_df = dataframe_svm_t[:, :]

#         df_ = dataframe_svm_t[:, list_param[0]]
#         svcc.fit(df_[train_ind], dataframe_svm_l[train_ind])
#         list_preds = list_model[0].predict_proba(df_[test_ind])
        list_preds = np.array()
        for i in range(len(list_model)):
            df_ = dataframe_svm_t[:, list_param[i]]
            list_model[i].fit(df_[train_ind], dataframe_svm_l[train_ind])
            list_preds = np.hstack((list_preds, list_model[i].predict_proba(df_[test_ind]))
            scipy.save(list_name[i], list_model[i].predict_proba(df_[test_ind]))
        
#         print(shape(list_labels_f))
#         if list_preds_f == []:
#             list_preds_f = list_preds
#             list_labels_f = dataframe_svm_l[test_ind]
#         else:
#             list_preds_f  = np.vstack((list_preds_f, list_preds))
          list_labels_f = np.hstack((list_labels_f, dataframe_svm_l[test_ind]))
#     return list_preds_f, list_labels_f

In [66]:
fin_classif([lrc, knc, svcc, rfc, xgbc], [load('regr_param.npy'), load('regr_param.npy'), load('regr_param.npy'), 
                               load('boost_param.npy'), load('boost_param.npy')],
                              ['lrc.npy', 'knc.npy', 'svcc.npy', 'rfc.npy', 'xgbc.npy',])
# print(shape(rl), shape(ll))

In [ ]:
lrc_ridge_scoring1 = cross_validation.cross_val_score(rfc, rl1, ll1,
                                                  scoring='f1_weighted', cv = 5)
print('lrc --  mean:{}, max:{}, min:{}, std:{}'.format(lrc_ridge_scoring1.mean(), lrc_ridge_scoring1.max(),
                                                     lrc_ridge_scoring1.min(), lrc_ridge_scoring1.std()))

In [23]:
from scipy import save
save('rl.npy', rl)
save('ll.npy', ll)